In [7]:
#https://towardsdatascience.com/solving-geographic-travelling-salesman-problems-using-python-e57284b14cd7
#Todo:
#Open Google Earth, get latitude, longitude, and name of the place
#get the libraries info of the distance of those places,
#turn the matrix of the distances into a simetric matrix 


In [8]:
!pip install routingpy concorde folium urlparse

ERROR: Could not find a version that satisfies the requirement urlparse (from versions: none)
ERROR: No matching distribution found for urlparse


In [14]:
import routingpy as rp
import numpy as np
import os
import folium

In [10]:
def symmetricize(m, high_int=None):
    
    # if high_int not provided, make it equal to 10 times the max value:
    if high_int is None:
        high_int = round(10*m.max())
        
    m_bar = m.copy()
    np.fill_diagonal(m_bar, 0)
    u = np.matrix(np.ones(m.shape) * high_int)
    np.fill_diagonal(u, 0)
    m_symm_top = np.concatenate((u, np.transpose(m_bar)), axis=1)
    m_symm_bottom = np.concatenate((m_bar, u), axis=1)
    m_symm = np.concatenate((m_symm_top, m_symm_bottom), axis=0)
    
    return m_symm.astype(int) # Concorde requires integer weights

In [11]:
def solve_concorde(matrix):
    problem = Problem.from_matrix(matrix)
    solution = run_concorde(problem)
    print(solution.tour)
    return solution

In [12]:
class GeographicTSP:
    
    def __init__(self, points, profile):
        
        if isinstance(points[0], list) or isinstance(points[0], tuple):
            # List of (lon, lat) pairs
            self.points = points
            self.names = None
            
        elif isinstance(points[0], dict):
            # List of dicts of form {'name': xxx, 'lon': yyy, 'lat': zzz}
            self.points = [(p['lon'], p['lat']) for p in points]
            self.names = [p['name'] for p in points]
        else:
            raise ValueError("Invalid input format. Expected list of (lon, lat) tuples or dictionaries.")

        self.length = len(points)
        self.profile = profile
        
    def solve(self, api):
        
        matrix = api.matrix(locations=self.points, profile=self.profile)
        durations = np.matrix(matrix.durations)
        
        # test if durations is symmetric
        if np.array_equal(durations, durations.transpose()):
            # it is symmetric, do nothing
            print('distance matrix is symmetric')
            pass
        else:
            print('distance matrix is not symmetric; making it so')
            durations = symmetricize(durations)
            
        solution = solve_concorde(durations)
        
        if len(solution.tour) == self.length:
            return solution.tour
        else: 
            # check that alternate elements of solution.tour are the original points
            if max(solution.tour[0::2]) == self.length-1:
                # alternate elements (starting at index 0) are original
                self.tour = solution.tour[0::2]
                return self.tour
            else:
                # alternate elements (starting at index 1) are original
                self.tour = solution.tour[1::2]
                return self.tour
            
    def get_directions(self, api):
        
        try:
            points_ordered = [self.points[i] for i in self.tour]
            self.points_ordered = points_ordered
            if self.names is not None:
                names_ordered = [self.names[i] for i in self.tour]
                self.names_ordered = names_ordered
        except AttributeError:
            print("self.tour does not exist; ensure solve() is run first")
            
        points_ordered_with_return = points_ordered + [points_ordered[0]]
        
        directions = api.directions(locations=points_ordered_with_return, profile=self.profile)
        self.directions = directions
        return self.directions
    
    def generate_map(self):
        # Create a map centered at a specific location
        route_points = [(y, x) for (x, y) in self.points_ordered]
        centre = np.mean([x for (x, y) in route_points]), np.mean([y for (x, y) in route_points])
        
        try:
            route_line = [(y, x) for (x, y) in self.directions.geometry] # folium needs lat, long
        except AttributeError:
            print("self.directions does not exist; ensure get_directions() is run first")
        
        m = folium.Map(location=centre, zoom_start=12, zoom_control=False)

        # Create a feature group for the route line
        route_line_group = folium.FeatureGroup(name='Route Line')

        # Add the route line to the feature group
        folium.PolyLine(route_line, color='red', weight=2).add_to(route_line_group)

        # Add the feature group to the map
        route_line_group.add_to(m)

        # Create a feature group for the route points
        route_points_group = folium.FeatureGroup(name='Route Points')

        # Add the route points to the feature group
        if self.names is None:
            names = route_points
        else:
            names = self.names_ordered
        for i, (point, name) in enumerate(zip(route_points, names)):
            folium.Marker(location=point, tooltip=f'{i}: {name}').add_to(route_points_group)

        # Add the feature group to the map
        route_points_group.add_to(m)

        # Create a custom tile layer with a partially greyed out basemap
        custom_tile_layer = folium.TileLayer(
            tiles='http://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png',
            attr='CartoDB Positron',
            name='Positron',
            overlay=True,
            control=True,
            opacity=0.7  # Adjust opacity to control the level of greying out
        )

        # Add the custom tile layer to the map
        custom_tile_layer.add_to(m)

        # Add layer control to the map
        folium.LayerControl().add_to(m)

        self.map = m
        return m

In [15]:
coordinates = [
                [-1.8162, 53.3651],
                [-1.8764, 53.3973],
                [-1.8757, 53.3630],
                [-1.7714, 53.3649],
                [-1.9098, 53.3578],
                [-1.9173, 53.3637],
                [-1.8826, 53.3803],
                [-1.7963, 53.3893],
                [-1.8096, 53.3492]
              ]

edale = GeographicTSP(points=coordinates, profile='hike')

In [26]:
import configparser
config = configparser.ConfigParser()
config.read('.config')
api_key = config.get('API', 'GRAPHHOPPER_API_KEY')

In [27]:
#api_key = os.environ['GRAPHHOPPER_API_KEY'] # get a free key at https://www.graphhopper.com/
api = rp.Graphhopper(api_key=api_key)

In [ ]:
tour = edale.solve(api=api)

In [ ]:
tour

In [ ]:
edale.get_directions(api=api)

In [ ]:
edale.generate_map()

In [ ]:
edale.map.save('edale.html')

In [17]:
!pip install pykml

     ---------------------------------------- 0.0/41.1 kB ? eta -:--:--
     ------------------- -------------------- 20.5/41.1 kB 640.0 kB/s eta 0:00:01
     ---------------------------------------- 41.1/41.1 kB 490.2 kB/s eta 0:00:00
  Obtaining dependency information for lxml>=3.3.6 from https://files.pythonhosted.org/packages/31/58/e3b3dd6bb2ab7404f1f4992e2d0e6926ed40cef8ce1b3bbefd95877499e1/lxml-4.9.3-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   -- ------------------------------------- 0.2/3.8 MB 4.7 MB/s eta 0:00:01
   ------- -------------------------------- 0.7/3.8 MB 7.5 MB/s eta 0:00:01
   ------------- -------------------------- 1.3/3.8 MB 8.9 MB/s eta 0:00:01
   -------------------- ------------------- 1.9/3.8 MB 10.3 MB/s eta 0:00:01
   ---------------------------- ----------- 2.7/3.8 MB 11.5 MB/s eta 0:00:01
   ------------------------------------- -- 3.5/3.8 MB 12.5 MB/s eta 0:00:01
   ------------------------

In [18]:
from pykml import parser

kml_file = os.path.join('london.kml')

with open(kml_file) as f:
  doc = parser.parse(f).getroot()

points = []

for x in doc.Document.Placemark:
    name = str(x.name)
    coords = str(x.Point.coordinates).split(',')
    lon = round(float(coords[0]), 4)
    lat = round(float(coords[1]), 4)
    points.append({'name': name, 'lon': lon, 'lat': lat})
    
points

FileNotFoundError: [Errno 2] No such file or directory: 'london.kml'

In [ ]:
london = GeographicTSP(points, profile='car')
london.points

In [ ]:
london.names

In [ ]:
london.solve(api)

In [ ]:
london.get_directions(api)

In [ ]:
london.generate_map()

In [ ]:
london.map.save('london.html')